In [11]:
import pandas

## functions & options


In [24]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/1)data_input/'
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

# 1] Data download

In [25]:
%%time
path = input_file_directory + "xenabrowser_brut_data/xena_surv_ATG7.tsv"
df_ori= pandas.read_csv(path, sep = "\t")
print(df_ori.shape)

(19131, 31)
Wall time: 102 ms


In [26]:
path = output_file_directory + "ATG7/TCGA_GTEX/all_tissues/"
path2 = 'dataframes_for_input/'

In [27]:
%%time
# DL data ATG7 isoform expression
df_ATG7_Normal = pandas.read_csv(path+path2+'df_ATG7_Normal_all.csv', sep = ",")
df_ATG7_Primary = pandas.read_csv(path+path2+'df_ATG7_Primary_all.csv', sep = ",")

#create df with the two groups
df_ATG7_NormPrim = pandas.concat([df_ATG7_Normal, df_ATG7_Primary], axis=0)
print(df_ATG7_NormPrim.shape)
df_ATG7_NormPrim.head(2)

(13869, 19)
Wall time: 63.9 ms


,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_tpm,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,GTEX-ZTTD-0326-SM-57WFW,Normal Tissue,Muscle,NaN,NaN,5.085097,3.955992,2.641679,1.137484,32.944283,14.519306,5.240574,1.19997,20.959850,69.271996,25.002916,5.725088,0.440723,0.159074
1,GTEX-OXRO-1726-SM-3LK6C,Normal Tissue,Muscle,NaN,NaN,4.001888,2.821767,1.823465,0.000000,15.020948,6.070277,2.539302,0.00000,8.609579,70.506084,29.493916,0.000000,0.404121,0.169051


In [28]:
#remove NaN value in OS column (will remove Normal Tissue).
df_ATG7_Prim = df_ATG7_NormPrim.dropna(axis=0)
print(df_ATG7_Prim.shape)
df_ATG7_Prim.head()

(7974, 19)


,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_tpm,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,TCGA-OR-A5J2-01,Primary Tumor,Adrenal Gland,1.0,1677.0,4.874006,3.384104,2.244552,1.761557,28.323920,9.440395,3.738901,2.390640,15.569935,60.632203,24.013593,15.354204,0.333301,0.132005
1,TCGA-P6-A5OG-01,Primary Tumor,Adrenal Gland,1.0,383.0,4.841178,4.140146,1.686916,0.000000,27.664191,16.632271,2.219678,0.000000,18.851949,88.225739,11.774261,0.000000,0.601220,0.080236
2,TCGA-OR-A5J8-01,Primary Tumor,Adrenal Gland,1.0,579.0,4.816092,4.629437,0.400624,0.000000,27.170080,23.751377,0.320079,0.000000,24.071456,98.670297,1.329703,0.000000,0.874174,0.011781
3,TCGA-OR-A5K5-01,Primary Tumor,Adrenal Gland,1.0,498.0,4.803555,4.291540,0.201684,0.056593,26.926343,18.583140,0.150040,0.040007,18.773186,98.987671,0.799224,0.213105,0.690147,0.005572
4,TCGA-PA-A5YG-01,Primary Tumor,Adrenal Gland,0.0,756.0,4.658238,3.223226,2.978529,1.669034,24.250465,8.338726,6.881821,2.180015,17.400563,47.922164,39.549417,12.528419,0.343858,0.283781


In [29]:
#How many each organs are the data ? 
df_ATG7_Prim['Primary_Site'].value_counts()

Breast                     1003
Lung                        943
Kidney                      759
Brain                       634
Head And Neck Region        476
Thyroid Gland               464
Prostate                    412
Stomach                     391
Bladder                     338
Ovary                       324
Colon                       281
Cervix                      277
Soft Tissue,Bone            229
Liver                       180
Esophagus                   177
Pancreas                    162
Paraganglia                 150
Endometrium                 140
Testis                      115
Thymus                       90
Rectum                       88
Skin                         77
Lining Of Body Cavities      75
Adrenal Gland                52
Uterus                       45
Eye                          36
Bile Duct                    31
Lymphatic Tissue             25
Name: Primary_Site, dtype: int64

# 2] All organs High/Low expression per Median for Isoform expression

In [30]:
# What is the median for iso 1 and iso 2? If I want to create two groups
#High >50% and low <50%

isoforms = [["ATG7", "total"], ["ATG7_1", "iso1"], ["ATG7_2", "iso2"]]

df_LoHi = [] 

for isoform in isoforms:
    print(isoform[0])
    xMed = df_ATG7_Prim[isoform[0]].median()
    print("median is", xMed)
    
    #create Low
    low = df_ATG7_Prim.loc[df_ATG7_Prim[isoform[0]] <= xMed].copy()
    low['L/H'] = "Low_"+isoform[1]
    #move the column
    col = low.pop('L/H')
    low.insert(3, 'L/H', col)
    
    #create High
    high = df_ATG7_Prim.loc[df_ATG7_Prim[isoform[0]] > xMed].copy()
    high['L/H'] = "High_"+isoform[1]
    #move the column
    col = high.pop('L/H')
    high.insert(3, 'L/H', col)
    
    #save
    df_LoHi.append(low)
    df_LoHi.append(high)
    print()
    
print(len(df_LoHi))

ATG7
median is 3.970031294649498

ATG7_1
median is 2.9785289937565813

ATG7_2
median is 1.7090630532755215

6


In [31]:
df_total_LoHi_prim = pandas.concat([df_LoHi[0], df_LoHi[1]], axis=0)
df_iso1_LoHi_prim = pandas.concat([df_LoHi[2], df_LoHi[3]], axis=0)
df_iso2_LoHi_prim = pandas.concat([df_LoHi[4], df_LoHi[5]], axis=0)
print(len(df_total_LoHi_prim))
print(len(df_iso1_LoHi_prim))
print(len(df_iso2_LoHi_prim))
df_iso2_LoHi_prim

7974
7974
7974


,sample,Sample_Type,Primary_Site,L/H,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_tpm,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
1,TCGA-P6-A5OG-01,Primary Tumor,Adrenal Gland,Low_iso2,1.0,383.0,4.841178,4.140146,1.686916,0.000000,27.664191,16.632271,2.219678,0.000000,18.851949,88.225739,11.774261,0.000000,0.601220,0.080236
2,TCGA-OR-A5J8-01,Primary Tumor,Adrenal Gland,Low_iso2,1.0,579.0,4.816092,4.629437,0.400624,0.000000,27.170080,23.751377,0.320079,0.000000,24.071456,98.670297,1.329703,0.000000,0.874174,0.011781
3,TCGA-OR-A5K5-01,Primary Tumor,Adrenal Gland,Low_iso2,1.0,498.0,4.803555,4.291540,0.201684,0.056593,26.926343,18.583140,0.150040,0.040007,18.773186,98.987671,0.799224,0.213105,0.690147,0.005572
9,TCGA-OR-A5K0-01,Primary Tumor,Adrenal Gland,Low_iso2,0.0,1029.0,4.157133,3.885007,0.464706,0.150597,16.841098,13.774187,0.380036,0.110028,14.264252,96.564386,2.664256,0.771357,0.817891,0.022566
10,TCGA-OR-A5K4-01,Primary Tumor,Adrenal Gland,Low_iso2,0.0,1082.0,4.156189,3.603709,1.490579,0.000000,16.829428,11.156949,1.810017,0.000000,12.966966,86.041323,13.958677,0.000000,0.662943,0.107551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7939,TCGA-AX-A3GI-01,Primary Tumor,Endometrium,High_iso2,0.0,12.0,3.701287,2.474798,1.748182,1.250954,12.007633,4.558893,2.359349,1.379987,8.298229,54.938145,28.431954,16.629901,0.379666,0.196487
7976,TCGA-KP-A3VZ-01,Primary Tumor,Endometrium,High_iso2,0.0,722.0,3.334459,2.538259,1.718103,0.000000,9.087234,4.808877,2.290036,0.000000,7.098913,67.741035,32.258965,0.000000,0.529190,0.252006
7986,TCGA-V4-A9F3-01,Primary Tumor,Eye,High_iso2,1.0,873.0,4.112805,2.555669,2.150693,1.618018,16.301259,4.879402,3.440411,2.069530,10.389342,46.965456,33.114805,19.919739,0.299327,0.211052
7994,TCGA-V4-A9EU-01,Primary Tumor,Eye,High_iso2,0.0,709.0,3.775297,2.639159,2.097507,0.251019,12.692341,5.229685,3.279692,0.190047,8.699424,60.115306,37.700101,2.184594,0.412035,0.258399


In [32]:
#save to csv
df_total_LoHi_prim.to_csv(path+path2+'df_total_LoHi_prim.tsv',sep = "\t", index=False)
df_iso1_LoHi_prim.to_csv(path+path2+'df_ATG7_iso1_LoHi_prim.tsv',sep = "\t", index=False)
df_iso2_LoHi_prim.to_csv(path+path2+'df_ATG7_iso2_LoHi_prim.tsv',sep = "\t", index=False)

# 3] All organs High/Low expression from ratio iso1/iso2

In [19]:
df_ATG7_Prim.head()

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,...,ATG7_3tpm,log2_p1_ATG7_total,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
7429,TCGA-OR-A5J2-01,Primary Tumor,Adrenal Gland,1.0,1677.0,4.824,3.239,1.903,1.258,28.323920,...,2.390640e+00,4.874006,3.384104,2.244552,15.569935,60.632203,24.013593,1.535420e+01,0.333301,0.132005
7430,TCGA-P6-A5OG-01,Primary Tumor,Adrenal Gland,1.0,383.0,4.790,4.056,1.151,-9.966,27.664191,...,-1.495113e-07,4.841178,4.140146,1.686916,18.851948,88.225740,11.774261,-7.930814e-07,0.601220,0.080236
7431,TCGA-OR-A5J8-01,Primary Tumor,Adrenal Gland,1.0,579.0,4.764,4.570,-1.639,-9.966,27.170080,...,-1.495113e-07,4.816092,4.629437,0.400624,24.071456,98.670297,1.329703,-6.211145e-07,0.874174,0.011781
7432,TCGA-OR-A5K5-01,Primary Tumor,Adrenal Gland,1.0,498.0,4.751,4.216,-2.727,-4.608,26.926343,...,4.000660e-02,4.803555,4.291540,0.201684,18.773186,98.987671,0.799224,2.131050e-01,0.690147,0.005572
7433,TCGA-PA-A5YG-01,Primary Tumor,Adrenal Gland,0.0,756.0,4.600,3.060,2.783,1.125,24.250465,...,2.180015e+00,4.658238,3.223226,2.978529,17.400563,47.922164,39.549417,1.252842e+01,0.343858,0.283781


In [25]:
df_ATG7_Prim = df_ATG7_Prim.assign(ratio_1_2 = (df_ATG7_Prim['ATG7_1tpm'] / df_ATG7_Prim['ATG7_2tpm']),
                                   ratio_2_1 = (df_ATG7_Prim['ATG7_2tpm'] / df_ATG7_Prim['ATG7_1tpm']))
df_ATG7_Prim.head()

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,...,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm,ratio_1_2,ratio_2_1
7429,TCGA-OR-A5J2-01,Primary Tumor,Adrenal Gland,1.0,1677.0,4.824,3.239,1.903,1.258,28.323920,...,3.384104,2.244552,15.569935,60.632203,24.013593,1.535420e+01,0.333301,0.132005,2.524912,0.396053
7430,TCGA-P6-A5OG-01,Primary Tumor,Adrenal Gland,1.0,383.0,4.790,4.056,1.151,-9.966,27.664191,...,4.140146,1.686916,18.851948,88.225740,11.774261,-7.930814e-07,0.601220,0.080236,7.493102,0.133456
7431,TCGA-OR-A5J8-01,Primary Tumor,Adrenal Gland,1.0,579.0,4.764,4.570,-1.639,-9.966,27.170080,...,4.629437,0.400624,24.071456,98.670297,1.329703,-6.211145e-07,0.874174,0.011781,74.204745,0.013476
7432,TCGA-OR-A5K5-01,Primary Tumor,Adrenal Gland,1.0,498.0,4.751,4.216,-2.727,-4.608,26.926343,...,4.291540,0.201684,18.773186,98.987671,0.799224,2.131050e-01,0.690147,0.005572,123.854793,0.008074
7433,TCGA-PA-A5YG-01,Primary Tumor,Adrenal Gland,0.0,756.0,4.600,3.060,2.783,1.125,24.250465,...,3.223226,2.978529,17.400563,47.922164,39.549417,1.252842e+01,0.343858,0.283781,1.211703,0.825284


In [26]:
# What is the median for iso 1 and iso 2? If I want to create two groups
#High >50% and low <50%

ratios = [["ratio_1_2", "iso1/iso2"], ["ratio_2_1", "iso2/iso1"]]

df_LoHi_ratio = [] 

for ratio in ratios:
    print(ratio[0])
    xMed = df_ATG7_Prim[ratio[0]].median()
    print("median is", xMed)
    
    #create Low
    low = df_ATG7_Prim.loc[df_ATG7_Prim[ratio[0]] <= xMed].copy()
    low['L/H'] = "Low_"+ratio[1]
    #move the column
    col = low.pop('L/H')
    low.insert(3, 'L/H', col)
    
    #create High
    high = df_ATG7_Prim.loc[df_ATG7_Prim[ratio[0]] > xMed].copy()
    high['L/H'] = "High_"+ratio[1]
    #move the column
    col = high.pop('L/H')
    high.insert(3, 'L/H', col)
    
    #save
    df_LoHi_ratio.append(low)
    df_LoHi_ratio.append(high)
    print()
    
print(len(df_LoHi_ratio))

ratio_1_2
median is 2.658007960737473

ratio_2_1
median is 0.28967539704320944

4


In [28]:
df_ratio1_2_LoHi_prim = pandas.concat([df_LoHi_ratio[0], df_LoHi_ratio[1]], axis=0)
df_ratio2_1_LoHi_prim = pandas.concat([df_LoHi_ratio[2], df_LoHi_ratio[3]], axis=0)
print(len(df_ratio1_2_LoHi_prim))
print(len(df_ratio2_1_LoHi_prim))
df_ratio2_1_LoHi_prim

9130
9130


,sample,Sample_Type,Primary_Site,L/H,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,...,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm,ratio_1_2,ratio_2_1
7430,TCGA-P6-A5OG-01,Primary Tumor,Adrenal Gland,Low_iso2/iso1,1.0,383.0,4.790,4.056,1.1510,-9.9660,...,4.140146,1.686916,18.851948,88.225740,11.774261,-7.930814e-07,0.601220,0.080236,7.493102,0.133456
7431,TCGA-OR-A5J8-01,Primary Tumor,Adrenal Gland,Low_iso2/iso1,1.0,579.0,4.764,4.570,-1.6390,-9.9660,...,4.629437,0.400624,24.071456,98.670297,1.329703,-6.211145e-07,0.874174,0.011781,74.204745,0.013476
7432,TCGA-OR-A5K5-01,Primary Tumor,Adrenal Gland,Low_iso2/iso1,1.0,498.0,4.751,4.216,-2.7270,-4.6080,...,4.291540,0.201684,18.773186,98.987671,0.799224,2.131050e-01,0.690147,0.005572,123.854793,0.008074
7436,TCGA-OR-A5JZ-01,Primary Tumor,Adrenal Gland,Low_iso2/iso1,0.0,822.0,4.106,3.386,1.4600,-9.9660,...,3.517669,1.906923,13.203206,79.171092,20.828909,-1.132386e-06,0.607075,0.159714,3.801020,0.263087
7437,TCGA-OR-A5JR-01,Primary Tumor,Adrenal Gland,Low_iso2/iso1,0.0,3688.0,4.097,3.231,1.4390,0.8805,...,3.376872,1.891547,13.938527,67.354213,19.444871,1.320092e+01,0.548640,0.158390,3.463855,0.288696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16585,TCGA-V4-A9ET-01,Primary Tumor,Eye,High_iso2/iso1,0.0,1362.0,3.180,1.688,1.1450,-0.0725,...,2.077618,1.682781,6.381547,50.475177,34.638329,1.488649e+01,0.355448,0.243925,1.457206,0.686245
16588,TCGA-V4-A9EQ-01,Primary Tumor,Eye,High_iso2/iso1,1.0,455.0,3.128,1.532,0.8961,0.3346,...,1.960091,1.516029,6.010921,48.093541,30.944150,2.096231e+01,0.330716,0.212788,1.554205,0.643416
16591,TCGA-VD-AA8P-01,Primary Tumor,Eye,High_iso2/iso1,0.0,86.0,3.057,1.637,0.9191,0.0014,...,2.038852,1.531037,5.999090,51.827593,31.503698,1.666871e+01,0.373637,0.227117,1.645127,0.607856
16597,TCGA-V4-A9EY-01,Primary Tumor,Eye,High_iso2/iso1,0.0,837.0,2.892,1.840,0.7999,-1.6390,...,2.195064,1.454166,5.639160,63.468681,30.855314,5.676004e+00,0.482229,0.234436,2.056977,0.486150


In [29]:
#save to csv
path = 'ATG7/dataframes_for_input/'
df_ratio1_2_LoHi_prim.to_csv(output_file_directory+path+'df_ATG7_ratio1_2_LoHi_prim.tsv',sep = "\t", index=False)
df_ratio2_1_LoHi_prim.to_csv(output_file_directory+path+'df_ATG7_ratio2_1_LoHi_prim.tsv',sep = "\t", index=False)